# Description

This notebook projects the PhenomeXcan data (MultiXcan results) into the MultiPLIER latent space. It takes only the top 1% of genes from each latent variable.

*Technical debt:* the notebook is usable and does its job, but will be refactored in the future, since other configurations are also useful (for instance, take different percentages of top genes to make the projection). Ideas for future refactoring:

1. Move projection code into libs.

# Modules loadings

In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import os
import pickle

import numpy as np
import pandas as pd

import settings as conf
from metadata import GENE_ID_TO_NAME_MAP, GENE_NAME_TO_ID_MAP

# Read MultiPLIER model metadata

In [3]:
metadata_dir = os.path.dirname(conf.MULTIPLIER_SETTINGS['RECOUNT2_FULL_MODEL_FILE'])
metadata_file = os.path.join(metadata_dir, 'multiplier_model_metadata.pkl')
display(metadata_file)

'/media/miltondp/Elements/projects/multiplier/recount2_PLIER_data/multiplier_model_metadata.pkl'

In [4]:
with open(metadata_file, 'rb') as handle:
    multiplier_model_metadata = pickle.load(handle)

In [5]:
multiplier_model_metadata

{'L1': 120.5660870071812, 'L2': 241.1321740143624, 'L3': 0.012696760842460974}

# Read MultiPLIER Z (loadings)

In [6]:
multiplier_model_z = pd.read_pickle(os.path.join(conf.MULTIPLIER_SETTINGS['RECOUNT2_DATA_DIR'], 'multiplier_model_z.pkl'))

In [7]:
multiplier_model_z.shape

(6750, 987)

In [8]:
multiplier_model_z.head()

,LV1,LV2,LV3,LV4,LV5,LV6,LV7,LV8,LV9,LV10,...,LV978,LV979,LV980,LV981,LV982,LV983,LV984,LV985,LV986,LV987
GAS6,0.000000,0.0,0.039438,0.0,0.050476,0.000000,0.0,0.000000,0.590949,0.000000,...,0.050125,0.00000,0.033407,0.000000,0.000000,0.005963,0.347362,0.0,0.000000,0.000000
MMP14,0.000000,0.0,0.000000,0.0,0.070072,0.000000,0.0,0.004904,1.720179,2.423595,...,0.000000,0.00000,0.001007,0.000000,0.035747,0.000000,0.000000,0.0,0.014978,0.000000
DSP,0.000000,0.0,0.000000,0.0,0.000000,0.041697,0.0,0.005718,0.000000,0.000000,...,0.020853,0.00000,0.000000,0.000000,0.000000,0.005774,0.000000,0.0,0.000000,0.416405
MARCKSL1,0.305212,0.0,0.000000,0.0,0.000000,0.000000,0.0,0.000000,0.161843,0.149471,...,0.027134,0.05272,0.000000,0.030189,0.060884,0.000000,0.000000,0.0,0.000000,0.448480
SPARC,0.000000,0.0,0.000000,0.0,0.000000,0.000000,0.0,0.000000,0.000000,0.014014,...,0.000000,0.00000,0.000000,0.000000,0.000000,0.000000,0.067779,0.0,0.122417,0.062665


# Load PhenomeXcan data for S-MultiXcan results

In [9]:
smultixcan_results_filename = os.path.join(conf.PHENOMEXCAN_SETTINGS['GENE_ASSOC_DIR'], 'smultixcan-mashr-zscores.pkl')
display(smultixcan_results_filename)

'/media/miltondp/Elements/projects/phenomexcan/base/gene_assoc/smultixcan-mashr-zscores.pkl'

In [10]:
smultixcan_results = pd.read_pickle(smultixcan_results_filename)

In [11]:
smultixcan_results.shape

(22515, 4091)

In [12]:
smultixcan_results.head()

,20096_1-Size_of_red_wine_glass_drunk_small_125ml,2345-Ever_had_bowel_cancer_screening,N49-Diagnoses_main_ICD10_N49_Inflammatory_disorders_of_male_genital_organs_not_elsewhere_classified,100011_raw-Iron,5221-Index_of_best_refractometry_result_right,20003_1141150624-Treatmentmedication_code_zomig_25mg_tablet,S69-Diagnoses_main_ICD10_S69_Other_and_unspecified_injuries_of_wrist_and_hand,20024_1136-Job_code_deduced_Information_and_communication_technology_managers,20002_1385-Noncancer_illness_code_selfreported_allergy_or_anaphylactic_reaction_to_food,G6_SLEEPAPNO-Sleep_apnoea,...,Astle_et_al_2016_Sum_basophil_neutrophil_counts,RA_OKADA_TRANS_ETHNIC,pgc.scz2,PGC_ADHD_EUR_2017,MAGIC_FastingGlucose,Astle_et_al_2016_Red_blood_cell_count,SSGAC_Depressive_Symptoms,BCAC_ER_positive_BreastCancer_EUR,IBD.EUR.Inflammatory_Bowel_Disease,Astle_et_al_2016_High_light_scatter_reticulocyte_count
gene_name,,,,,,,,,,,,,,,,,,,,,
ENSG00000000419,0.169468,0.102558,0.239545,0.887758,1.313448,1.472148,0.726160,1.516367,1.299771,1.068093,...,0.813014,0.275993,0.510834,0.024717,0.430951,0.824314,0.367414,1.377624,0.738444,0.298259
ENSG00000000457,1.358856,1.846875,0.139324,0.129530,0.757757,1.103979,0.612418,1.822327,2.035372,1.008058,...,1.441795,0.654791,2.545653,1.202984,0.514244,0.237223,0.414171,0.101731,1.012735,0.945167
ENSG00000000460,0.151008,1.173202,1.179426,0.571656,0.098771,0.221072,0.276415,0.461381,0.855502,0.201876,...,0.668962,0.300040,0.541782,1.033308,0.482261,0.695624,0.336480,0.083316,3.493196,0.991948
ENSG00000000938,1.302722,0.841524,1.578926,0.721340,0.139314,4.387016,0.125959,1.247123,0.215124,0.892083,...,0.126657,0.048048,1.886356,0.540496,0.127524,1.494501,0.056432,1.704863,1.351619,1.027297
ENSG00000000971,1.338813,0.262339,0.689379,1.702019,0.325859,0.063161,1.141126,0.882682,0.035533,1.810191,...,0.858497,1.675562,2.319072,1.598721,0.162958,0.005703,3.004544,0.803669,0.444266,0.165671


# Get genes in common

In [13]:
lvs_gene_ids = [
    GENE_NAME_TO_ID_MAP[g]
    for g in multiplier_model_z.index
    if g in GENE_NAME_TO_ID_MAP and GENE_NAME_TO_ID_MAP[g] in smultixcan_results.index
]

In [14]:
len(lvs_gene_ids)

6452

In [15]:
lvs_gene_names = [GENE_ID_TO_NAME_MAP[g] for g in lvs_gene_ids]

In [16]:
len(lvs_gene_names)

6452

In [17]:
assert len(lvs_gene_ids) == len(lvs_gene_names)

## MultiPLIER Z: select common genes and rename them to IDs

In [18]:
multiplier_model_z_common = multiplier_model_z.loc[lvs_gene_names].rename(index=GENE_NAME_TO_ID_MAP)

In [19]:
multiplier_model_z_common.shape

(6452, 987)

In [20]:
multiplier_model_z_common.head()

,LV1,LV2,LV3,LV4,LV5,LV6,LV7,LV8,LV9,LV10,...,LV978,LV979,LV980,LV981,LV982,LV983,LV984,LV985,LV986,LV987
ENSG00000183087,0.000000,0.0,0.039438,0.0,0.050476,0.000000,0.0,0.000000,0.590949,0.000000,...,0.050125,0.00000,0.033407,0.000000,0.000000,0.005963,0.347362,0.0,0.000000,0.000000
ENSG00000157227,0.000000,0.0,0.000000,0.0,0.070072,0.000000,0.0,0.004904,1.720179,2.423595,...,0.000000,0.00000,0.001007,0.000000,0.035747,0.000000,0.000000,0.0,0.014978,0.000000
ENSG00000096696,0.000000,0.0,0.000000,0.0,0.000000,0.041697,0.0,0.005718,0.000000,0.000000,...,0.020853,0.00000,0.000000,0.000000,0.000000,0.005774,0.000000,0.0,0.000000,0.416405
ENSG00000175130,0.305212,0.0,0.000000,0.0,0.000000,0.000000,0.0,0.000000,0.161843,0.149471,...,0.027134,0.05272,0.000000,0.030189,0.060884,0.000000,0.000000,0.0,0.000000,0.448480
ENSG00000113140,0.000000,0.0,0.000000,0.0,0.000000,0.000000,0.0,0.000000,0.000000,0.014014,...,0.000000,0.00000,0.000000,0.000000,0.000000,0.000000,0.067779,0.0,0.122417,0.062665


## Quick look at LV603 genes

In [21]:
t = multiplier_model_z_common['LV603']

In [22]:
t.quantile(0.90)

0.38182870346382053

In [23]:
t.sort_values(ascending=False).head()

ENSG00000180871    5.320459
ENSG00000162747    5.128372
ENSG00000173535    5.035457
ENSG00000112303    4.680865
ENSG00000204160    4.495976
Name: LV603, dtype: float64

In [24]:
t = t[t > 0]

In [25]:
t.sort_values(ascending=False).rename(index=GENE_ID_TO_NAME_MAP).head(10)

CXCR2        5.320459
FCGR3B       5.128372
TNFRSF10C    5.035457
VNN2         4.680865
ZDHHC18      4.495976
MNDA         4.488505
CXCR1        4.442062
P2RY13       4.404405
VNN3         4.253184
FPR2         4.187560
Name: LV603, dtype: float64

# MultiPLIER Z: keep only those genes with high weight

In [26]:
PERCENTILE_SELECTED = 0.99

In [27]:
def zero_nonimportant_genes(x, perc=PERCENTILE_SELECTED):
#     x = x.copy()
    x_gt_zero = x[x > 0]
    q = x_gt_zero.quantile(perc)
    x[x < q] = 0.0
    return x

In [28]:
multiplier_model_z_common_important_genes = multiplier_model_z_common.copy().apply(zero_nonimportant_genes)

In [29]:
multiplier_model_z_common_important_genes.shape

(6452, 987)

In [30]:
multiplier_model_z_common_important_genes.head()

,LV1,LV2,LV3,LV4,LV5,LV6,LV7,LV8,LV9,LV10,...,LV978,LV979,LV980,LV981,LV982,LV983,LV984,LV985,LV986,LV987
ENSG00000183087,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
ENSG00000157227,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
ENSG00000096696,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
ENSG00000175130,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
ENSG00000113140,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [31]:
multiplier_model_z_common_important_genes.describe()

,LV1,LV2,LV3,LV4,LV5,LV6,LV7,LV8,LV9,LV10,...,LV978,LV979,LV980,LV981,LV982,LV983,LV984,LV985,LV986,LV987
count,6452.000000,6452.000000,6452.000000,6452.000000,6452.000000,6452.000000,6452.000000,6452.000000,6452.000000,6452.000000,...,6452.000000,6452.000000,6452.000000,6452.000000,6452.000000,6452.000000,6452.000000,6452.000000,6452.000000,6452.000000
mean,0.000463,0.010180,0.014661,0.011473,0.005909,0.004569,0.011694,0.017080,0.016936,0.012620,...,0.006355,0.007879,0.006136,0.007588,0.005837,0.006172,0.015327,0.012379,0.009219,0.010752
std,0.037218,0.196579,0.232178,0.173251,0.125828,0.081948,0.207472,0.284127,0.228894,0.236841,...,0.107570,0.135580,0.104844,0.148526,0.131026,0.128982,0.185517,0.184729,0.151214,0.136551
min,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
50%,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
75%,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
max,2.989508,5.695377,4.631069,3.820836,4.682360,2.431865,5.125221,6.110712,4.178663,7.058765,...,4.597933,6.467211,3.185780,6.472468,7.383183,6.872569,3.523630,4.717425,4.486173,2.537906


### Some testing on LV603

In [32]:
_lv_prev = multiplier_model_z_common['LV603']

In [33]:
_lv_prev.shape

(6452,)

In [34]:
_lv_prev_nonzero = _lv_prev[_lv_prev > 0.0]

In [35]:
_lv_prev_nonzero.shape

(2179,)

In [36]:
_lv_prev_q = _lv_prev_nonzero.quantile(PERCENTILE_SELECTED)
display(_lv_prev_q)

3.5563536352038447

In [37]:
_tmp = _lv_prev[_lv_prev > _lv_prev_q].shape
display(_tmp)

(22,)

In [38]:
assert _tmp[0] == 22

How many non-zero genes are in any LV? There must be around 22 (1 percent of nonzero loadings genes)

In [39]:
_any_lv = multiplier_model_z_common_important_genes['LV603']

In [40]:
_any_lv.shape

(6452,)

In [41]:
_any_lv.head()

ENSG00000183087    0.0
ENSG00000157227    0.0
ENSG00000096696    0.0
ENSG00000175130    0.0
ENSG00000113140    0.0
Name: LV603, dtype: float64

In [42]:
_any_lv[_any_lv > 0].shape

(22,)

In [43]:
assert _tmp[0] == _any_lv[_any_lv > 0].shape[0]

### Some testing on LV136

In [44]:
_any_lv = multiplier_model_z_common_important_genes['LV136']

In [45]:
_any_lv.shape

(6452,)

In [46]:
_any_lv.head()

ENSG00000183087    0.0
ENSG00000157227    0.0
ENSG00000096696    0.0
ENSG00000175130    0.0
ENSG00000113140    0.0
Name: LV136, dtype: float64

In [47]:
_any_lv[_any_lv > 0].shape

(20,)

### What's the coverage of important genes?

In [48]:
genes_covered = set()

for lv_col in multiplier_model_z_common_important_genes.columns:
    lv_values = multiplier_model_z_common_important_genes[lv_col]
    lv_values = lv_values[lv_values > 0.0]
    genes_covered.update(lv_values.index.tolist())

In [49]:
len(genes_covered)

6376

In [50]:
len(genes_covered) / multiplier_model_z_common_important_genes.shape[0]

0.9882207067575945

Even considering only the top 1% of genes in LVs, we still have that most genes considered are part of some LV.

## How many genes, on average, are nonzero for each LV?

In [51]:
multiplier_model_z_common_important_genes.head()

,LV1,LV2,LV3,LV4,LV5,LV6,LV7,LV8,LV9,LV10,...,LV978,LV979,LV980,LV981,LV982,LV983,LV984,LV985,LV986,LV987
ENSG00000183087,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
ENSG00000157227,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
ENSG00000096696,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
ENSG00000175130,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
ENSG00000113140,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [52]:
_tmp = multiplier_model_z_common_important_genes.apply(lambda x: x[x > 0].shape[0])

In [53]:
_tmp.describe()

count    987.000000
mean      27.256332
std        5.074710
min        1.000000
25%       25.000000
50%       27.000000
75%       30.000000
max       53.000000
dtype: float64

## Save MultiPLIER Z with only important genes selected

In [54]:
suffix = f'{PERCENTILE_SELECTED}'.replace('.', '_')
display(suffix)

'0_99'

In [55]:
output_filename = os.path.join(conf.LVS_MULTIPLIER_PHENOMEXCAN_SETTINGS['LVS_PROJECTIONS'], f'multiplier_z_important_genes_{suffix}.pkl')
display(output_filename)

'/media/miltondp/Elements/projects/k99/base/results/lvs_x_phenomexcan/projections/multiplier_z_important_genes_0_99.pkl'

In [56]:
multiplier_model_z_common_important_genes.to_pickle(output_filename)

## S-MultiXcan: keep common genes

In [57]:
smultixcan_results_common = smultixcan_results.loc[multiplier_model_z_common_important_genes.index]

In [58]:
smultixcan_results_common.shape

(6452, 4091)

In [59]:
smultixcan_results_common.head()

,20096_1-Size_of_red_wine_glass_drunk_small_125ml,2345-Ever_had_bowel_cancer_screening,N49-Diagnoses_main_ICD10_N49_Inflammatory_disorders_of_male_genital_organs_not_elsewhere_classified,100011_raw-Iron,5221-Index_of_best_refractometry_result_right,20003_1141150624-Treatmentmedication_code_zomig_25mg_tablet,S69-Diagnoses_main_ICD10_S69_Other_and_unspecified_injuries_of_wrist_and_hand,20024_1136-Job_code_deduced_Information_and_communication_technology_managers,20002_1385-Noncancer_illness_code_selfreported_allergy_or_anaphylactic_reaction_to_food,G6_SLEEPAPNO-Sleep_apnoea,...,Astle_et_al_2016_Sum_basophil_neutrophil_counts,RA_OKADA_TRANS_ETHNIC,pgc.scz2,PGC_ADHD_EUR_2017,MAGIC_FastingGlucose,Astle_et_al_2016_Red_blood_cell_count,SSGAC_Depressive_Symptoms,BCAC_ER_positive_BreastCancer_EUR,IBD.EUR.Inflammatory_Bowel_Disease,Astle_et_al_2016_High_light_scatter_reticulocyte_count
ENSG00000183087,0.322338,0.914191,0.257946,0.275937,1.144540,0.160631,2.456030,0.415369,0.398203,0.576533,...,0.118405,1.020714,1.071265,0.627512,0.016345,5.190657,0.142748,0.696664,0.359041,0.536451
ENSG00000157227,0.419139,0.968943,0.124378,0.237572,0.615571,0.379223,1.740254,0.639452,2.274270,1.293075,...,1.061050,0.318303,0.134067,0.291673,1.863238,1.134948,0.571293,1.031406,2.132095,1.700149
ENSG00000096696,1.128420,0.348985,0.653410,0.063173,0.413422,0.716294,0.076358,2.326302,0.544141,0.466012,...,0.530676,2.686939,0.122267,0.671125,1.188796,1.630909,1.007901,0.381546,1.364992,1.143065
ENSG00000175130,1.002128,0.021870,1.295205,0.290610,1.654534,0.647260,1.392347,0.903704,0.903001,0.262853,...,1.836999,0.377130,0.796055,0.423380,0.699479,1.114185,0.195246,1.577039,0.441571,0.141089
ENSG00000113140,0.981641,0.734060,0.058307,0.423182,2.159288,0.158809,1.230338,0.804094,1.148724,0.526070,...,0.272666,0.235158,1.421488,3.378371,0.010467,1.110320,0.714160,2.142983,0.893404,1.301364


# Multiply to obtains LVs associations to all traits

In [60]:
from numpy.linalg import pinv

In [61]:
# smultixcan_results_common_final = smultixcan_results_common_stded
smultixcan_results_common_final = smultixcan_results_common

In [62]:
# row z-score standardization
smultixcan_results_common_final = \
(
        # remove mean
        smultixcan_results_common_final.sub(smultixcan_results_common_final.mean(1), axis=0)
        # divide by std
        .div(smultixcan_results_common_final.std(1), axis=0)
)#.fillna(0)

In [63]:
# # row minmax standardization
# smultixcan_results_common_final = \
# (
#         # remove mean
#         smultixcan_results_common_final.sub(smultixcan_results_common_final.min(1), axis=0)
#         # divide by std
#         .div(smultixcan_results_common_final.max(1) - smultixcan_results_common_final.min(1), axis=0)
# )#.fillna(0)

In [64]:
# # row minmax total standardization
# smultixcan_results_common_final = \
# (
#         # remove mean
#         smultixcan_results_common_final.div(smultixcan_results_common_final.sum(1), axis=0)
# )#.fillna(0)

In [65]:
smultixcan_results_common_final.head(2)

,20096_1-Size_of_red_wine_glass_drunk_small_125ml,2345-Ever_had_bowel_cancer_screening,N49-Diagnoses_main_ICD10_N49_Inflammatory_disorders_of_male_genital_organs_not_elsewhere_classified,100011_raw-Iron,5221-Index_of_best_refractometry_result_right,20003_1141150624-Treatmentmedication_code_zomig_25mg_tablet,S69-Diagnoses_main_ICD10_S69_Other_and_unspecified_injuries_of_wrist_and_hand,20024_1136-Job_code_deduced_Information_and_communication_technology_managers,20002_1385-Noncancer_illness_code_selfreported_allergy_or_anaphylactic_reaction_to_food,G6_SLEEPAPNO-Sleep_apnoea,...,Astle_et_al_2016_Sum_basophil_neutrophil_counts,RA_OKADA_TRANS_ETHNIC,pgc.scz2,PGC_ADHD_EUR_2017,MAGIC_FastingGlucose,Astle_et_al_2016_Red_blood_cell_count,SSGAC_Depressive_Symptoms,BCAC_ER_positive_BreastCancer_EUR,IBD.EUR.Inflammatory_Bowel_Disease,Astle_et_al_2016_High_light_scatter_reticulocyte_count
ENSG00000183087,-0.745878,0.096334,-0.837508,-0.811906,0.424123,-0.975987,2.290384,-0.613494,-0.637921,-0.384155,...,-1.036076,0.247918,0.319853,-0.311612,-1.181309,6.181782,-1.001435,-0.213208,-0.693649,-0.441192
ENSG00000157227,-0.568076,0.186235,-0.972477,-0.817179,-0.298579,-0.622840,1.244446,-0.265815,1.977097,0.630932,...,0.312603,-0.706420,-0.959185,-0.742955,1.413176,0.413988,-0.359327,0.271932,1.782038,1.189423


In [66]:
_tmp = smultixcan_results_common_final.T.describe()

In [67]:
_tmp

,ENSG00000183087,ENSG00000157227,ENSG00000096696,ENSG00000175130,ENSG00000113140,ENSG00000117984,ENSG00000116016,ENSG00000129116,ENSG00000134686,ENSG00000108679,...,ENSG00000111716,ENSG00000166796,ENSG00000114331,ENSG00000131584,ENSG00000165410,ENSG00000172757,ENSG00000147862,ENSG00000008323,ENSG00000167083,ENSG00000149257
count,4.091000e+03,4.091000e+03,4.091000e+03,4.091000e+03,4.091000e+03,4.091000e+03,4.091000e+03,4.091000e+03,4.091000e+03,4.091000e+03,...,4.091000e+03,4.091000e+03,4.091000e+03,4.091000e+03,4.091000e+03,4.091000e+03,4.091000e+03,4.091000e+03,4.091000e+03,4.091000e+03
mean,2.431581e-17,2.475002e-17,-1.862765e-16,7.207901e-17,8.684218e-18,-7.468428e-17,3.994740e-17,-1.215791e-17,-8.944745e-17,-1.476317e-17,...,1.693423e-16,1.623949e-16,-3.300003e-17,-1.042106e-17,2.431581e-17,7.642112e-17,2.561844e-17,1.728159e-16,-1.102896e-16,-4.342109e-18
std,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,...,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00
min,-1.204328e+00,-1.143089e+00,-1.271972e+00,-1.317103e+00,-1.283498e+00,-1.320244e+00,-1.029331e+00,-1.316081e+00,-9.829798e-01,-1.310916e+00,...,-1.319291e+00,-1.333852e+00,-1.301824e+00,-1.156864e+00,-1.264905e+00,-1.088179e+00,-1.238807e+00,-1.290232e+00,-1.113326e+00,-1.122255e+00
25%,-7.486444e-01,-7.226303e-01,-7.495273e-01,-7.889164e-01,-8.041394e-01,-7.906524e-01,-6.222058e-01,-7.859982e-01,-6.238235e-01,-7.756578e-01,...,-7.778394e-01,-7.976366e-01,-7.892986e-01,-7.073513e-01,-7.579923e-01,-6.767003e-01,-7.742235e-01,-7.664349e-01,-6.765824e-01,-6.792223e-01
50%,-2.169492e-01,-2.222643e-01,-2.228226e-01,-2.030086e-01,-2.208637e-01,-2.200640e-01,-1.828274e-01,-2.021253e-01,-2.080361e-01,-2.239812e-01,...,-2.341722e-01,-1.955173e-01,-1.880682e-01,-1.936140e-01,-2.172526e-01,-2.327133e-01,-2.084080e-01,-2.023133e-01,-2.157647e-01,-2.226477e-01
75%,5.211107e-01,4.730266e-01,5.390974e-01,5.539437e-01,5.801046e-01,5.850646e-01,4.406687e-01,5.864414e-01,3.281706e-01,5.576957e-01,...,5.651602e-01,6.244570e-01,5.414168e-01,4.674506e-01,5.245787e-01,4.171930e-01,4.930067e-01,5.398479e-01,4.115888e-01,4.289178e-01
max,9.420778e+00,8.467100e+00,1.213591e+01,5.666027e+00,5.654056e+00,5.942638e+00,2.217474e+01,8.660557e+00,1.008603e+01,6.200707e+00,...,4.992684e+00,4.899750e+00,5.761907e+00,1.218858e+01,5.769775e+00,1.018035e+01,9.914657e+00,1.173137e+01,9.979559e+00,1.987825e+01


In [68]:
assert (_tmp.loc['mean'] < 1e-10).all()
assert np.allclose(_tmp.loc['std'].values, 1.0)

In [69]:
smultixcan_results_common_final = smultixcan_results_common_final.fillna(0)

In [70]:
multiplier_model_metadata['L2']

241.1321740143624

In [71]:
prev_mat = \
(
    multiplier_model_z_common_important_genes.T.dot(multiplier_model_z_common_important_genes)
    + (multiplier_model_metadata['L2'] * np.identity(multiplier_model_z_common_important_genes.shape[1]))
)

In [72]:
prev_mat.shape

(987, 987)

In [73]:
prev_mat = pd.DataFrame(pinv(prev_mat), index=prev_mat.index.copy(), columns=prev_mat.columns.copy())

In [74]:
lvs_traits_df = \
(
    prev_mat.dot(multiplier_model_z_common_important_genes.T).dot(smultixcan_results_common_final)
)

In [75]:
lvs_traits_df.shape

(987, 4091)

In [76]:
lvs_traits_df.head()

,20096_1-Size_of_red_wine_glass_drunk_small_125ml,2345-Ever_had_bowel_cancer_screening,N49-Diagnoses_main_ICD10_N49_Inflammatory_disorders_of_male_genital_organs_not_elsewhere_classified,100011_raw-Iron,5221-Index_of_best_refractometry_result_right,20003_1141150624-Treatmentmedication_code_zomig_25mg_tablet,S69-Diagnoses_main_ICD10_S69_Other_and_unspecified_injuries_of_wrist_and_hand,20024_1136-Job_code_deduced_Information_and_communication_technology_managers,20002_1385-Noncancer_illness_code_selfreported_allergy_or_anaphylactic_reaction_to_food,G6_SLEEPAPNO-Sleep_apnoea,...,Astle_et_al_2016_Sum_basophil_neutrophil_counts,RA_OKADA_TRANS_ETHNIC,pgc.scz2,PGC_ADHD_EUR_2017,MAGIC_FastingGlucose,Astle_et_al_2016_Red_blood_cell_count,SSGAC_Depressive_Symptoms,BCAC_ER_positive_BreastCancer_EUR,IBD.EUR.Inflammatory_Bowel_Disease,Astle_et_al_2016_High_light_scatter_reticulocyte_count
LV1,-0.011428,-0.003349,0.004548,0.001340,-0.008522,-0.012880,-0.010695,-0.008460,0.007192,0.000369,...,0.011257,-0.004408,-0.010043,-0.004443,-0.014030,-0.012685,0.025607,-0.011756,-0.009284,0.002057
LV2,-0.029025,-0.031933,0.035768,0.014905,-0.028030,-0.018171,-0.048591,-0.028944,0.035020,-0.022982,...,0.178994,-0.052794,0.080730,-0.010466,-0.030267,0.108304,-0.077986,-0.036163,0.086786,0.075796
LV3,-0.022593,0.041738,-0.052368,0.022981,-0.006197,-0.013814,0.007206,-0.018820,0.015699,-0.017102,...,0.083437,0.057528,0.113129,-0.027433,-0.043856,0.045791,-0.045977,-0.014493,0.014274,0.042264
LV4,0.032312,0.017544,0.049875,-0.007209,-0.008422,-0.003078,-0.012704,-0.003348,-0.011122,0.027604,...,0.059290,0.007098,0.082854,0.012517,-0.027010,0.082190,-0.000811,0.025551,-0.007174,0.052896
LV5,-0.020553,-0.031544,-0.029994,0.002896,0.023773,-0.018088,-0.009549,-0.033009,0.008574,-0.010971,...,-0.000531,-0.002261,0.002403,0.009198,-0.051971,0.077328,-0.010673,-0.036497,0.099145,0.047000


# Quick analysis

In [77]:
lvs_traits_df.loc['LV603'].sort_values(ascending=False).head(20)

30140_raw-Neutrophill_count                          0.552626
30000_raw-White_blood_cell_leukocyte_count           0.508006
30180_raw-Lymphocyte_percentage                      0.451405
30200_raw-Neutrophill_percentage                     0.430967
30100_raw-Mean_platelet_thrombocyte_volume           0.403399
Astle_et_al_2016_Myeloid_white_cell_count            0.402407
Astle_et_al_2016_Sum_neutrophil_eosinophil_counts    0.398964
Astle_et_al_2016_Granulocyte_count                   0.398326
30090_raw-Platelet_crit                              0.397491
Astle_et_al_2016_Sum_basophil_neutrophil_counts      0.392550
Astle_et_al_2016_Neutrophil_count                    0.391924
30190_raw-Monocyte_percentage                        0.384196
Astle_et_al_2016_White_blood_cell_count              0.382439
30150-Eosinophill_count                              0.367542
30130_raw-Monocyte_count                             0.360042
50_raw-Standing_height                               0.353690
30220_ra

In [78]:
lvs_traits_df.loc['LV136'].sort_values(ascending=False).head(20)

50_raw-Standing_height                                 0.400824
20015_raw-Sitting_height                               0.350660
5132_raw-3mm_strong_meridian_right                     0.296917
5099_raw-3mm_weak_meridian_right                       0.294336
5096_raw-3mm_weak_meridian_left                        0.293114
4079_raw-Diastolic_blood_pressure_automated_reading    0.288737
5134_raw-6mm_strong_meridian_left                      0.285232
5097_raw-6mm_weak_meridian_left                        0.279296
5098_raw-6mm_weak_meridian_right                       0.273827
5135_raw-3mm_strong_meridian_left                      0.262497
5133_raw-6mm_strong_meridian_right                     0.261074
23107_raw-Impedance_of_leg_right                       0.249909
23106_raw-Impedance_of_whole_body                      0.246306
23110_raw-Impedance_of_arm_left                        0.237503
23109_raw-Impedance_of_arm_right                       0.233388
30280_raw-Immature_reticulocyte_fraction

# Save

In [79]:
lvs_traits_df.shape

(987, 4091)

In [80]:
lvs_traits_df.head()

,20096_1-Size_of_red_wine_glass_drunk_small_125ml,2345-Ever_had_bowel_cancer_screening,N49-Diagnoses_main_ICD10_N49_Inflammatory_disorders_of_male_genital_organs_not_elsewhere_classified,100011_raw-Iron,5221-Index_of_best_refractometry_result_right,20003_1141150624-Treatmentmedication_code_zomig_25mg_tablet,S69-Diagnoses_main_ICD10_S69_Other_and_unspecified_injuries_of_wrist_and_hand,20024_1136-Job_code_deduced_Information_and_communication_technology_managers,20002_1385-Noncancer_illness_code_selfreported_allergy_or_anaphylactic_reaction_to_food,G6_SLEEPAPNO-Sleep_apnoea,...,Astle_et_al_2016_Sum_basophil_neutrophil_counts,RA_OKADA_TRANS_ETHNIC,pgc.scz2,PGC_ADHD_EUR_2017,MAGIC_FastingGlucose,Astle_et_al_2016_Red_blood_cell_count,SSGAC_Depressive_Symptoms,BCAC_ER_positive_BreastCancer_EUR,IBD.EUR.Inflammatory_Bowel_Disease,Astle_et_al_2016_High_light_scatter_reticulocyte_count
LV1,-0.011428,-0.003349,0.004548,0.001340,-0.008522,-0.012880,-0.010695,-0.008460,0.007192,0.000369,...,0.011257,-0.004408,-0.010043,-0.004443,-0.014030,-0.012685,0.025607,-0.011756,-0.009284,0.002057
LV2,-0.029025,-0.031933,0.035768,0.014905,-0.028030,-0.018171,-0.048591,-0.028944,0.035020,-0.022982,...,0.178994,-0.052794,0.080730,-0.010466,-0.030267,0.108304,-0.077986,-0.036163,0.086786,0.075796
LV3,-0.022593,0.041738,-0.052368,0.022981,-0.006197,-0.013814,0.007206,-0.018820,0.015699,-0.017102,...,0.083437,0.057528,0.113129,-0.027433,-0.043856,0.045791,-0.045977,-0.014493,0.014274,0.042264
LV4,0.032312,0.017544,0.049875,-0.007209,-0.008422,-0.003078,-0.012704,-0.003348,-0.011122,0.027604,...,0.059290,0.007098,0.082854,0.012517,-0.027010,0.082190,-0.000811,0.025551,-0.007174,0.052896
LV5,-0.020553,-0.031544,-0.029994,0.002896,0.023773,-0.018088,-0.009549,-0.033009,0.008574,-0.010971,...,-0.000531,-0.002261,0.002403,0.009198,-0.051971,0.077328,-0.010673,-0.036497,0.099145,0.047000


In [81]:
os.makedirs(conf.LVS_MULTIPLIER_PHENOMEXCAN_SETTINGS['LVS_PROJECTIONS'], exist_ok=True)

In [82]:
lvs_filename = os.path.join(conf.LVS_MULTIPLIER_PHENOMEXCAN_SETTINGS['LVS_PROJECTIONS'], 'lvs_x_smultixcan.pkl')
display(lvs_filename)

'/media/miltondp/Elements/projects/k99/base/results/lvs_x_phenomexcan/projections/lvs_x_smultixcan.pkl'

In [83]:
lvs_traits_df.to_pickle(lvs_filename)